In [111]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt

# GPT Variant Transformer

## 1. 기존 트랜스포머와 달라진 부분
* 인코더
    - 기존 트랜스포머의 인코더 모듈이 제외되었습니다.
    - 따라서 디코더 모듈 기반의 모델입니다.
* 디코더
    - 인코더 모듈이 제외됨에 따라 디코더 모듈의 Encoder-Decoder Attention 서브 레이어도 제외되었습니다.
* 포지션 정보 
    - 기존 삼감함수 기반의 포지셔널 인코딩 방식이 포지션 정보도 학습가능한 포지셔널 임베딩 레이어로 변경되었습니다.

## 2. 챗봇 데이터 로드 및 전처리

1) 한국어 데이터 셋

In [112]:
import os
from tensorflow import keras
import pandas as pd

chat_data_path = keras.utils.get_file(
    fname="chat_raw_data.csv",
    origin="https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
)

origin_data = pd.read_csv(chat_data_path, on_bad_lines='skip')
print(len(origin_data))
print(origin_data.head())

11823
                 Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0


2) 영어 데이터 셋

In [113]:
from tensorflow import keras

path_to_zip = keras.utils.get_file(
    'cornell_movie_dialogs.zip',
    origin='http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip',
    extract=True)

path_to_dataset = os.path.join(
    os.path.dirname(path_to_zip), "cornell movie-dialogs corpus")

path_to_movie_lines = os.path.join(path_to_dataset, 'movie_lines.txt')
path_to_movie_conversations = os.path.join(path_to_dataset,'movie_conversations.txt')

In [114]:
# 전처리 함수
def preprocess_sentence(sentence):
  # 입력받은 sentence를 소문자로 변경하고 양쪽 공백을 제거
  sentence = sentence.lower().strip()

  # 단어와 구두점(punctuation) 사이의 거리를 만듭니다.
  # 예를 들어서 "I am a student." => "I am a student ."와 같이
  # student와 온점 사이에 거리를 만듭니다.
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = re.sub(r'[" "]+', " ", sentence)

  # (a-z, A-Z, ".", "?", "!", ",")를 제외한 모든 문자를 공백인 ' '로 대체합니다.
  sentence = re.sub(r'[^가-힣0-9.,?!]', ' ', sentence)
  sentence = sentence.strip()
  return sentence

In [140]:
# 질문과 답변의 쌍인 데이터셋을 구성하기 위한 데이터 로드 함수

MAX_SAMPLES = 50_000

def load_conversations():
  id2line = {}
  with open(path_to_movie_lines, errors='ignore') as file:
    lines = file.readlines()
  for line in lines:
    parts = line.replace('\n', '').split(' +++$+++ ')
    id2line[parts[0]] = parts[4]

  inputs, outputs = [], []
  with open(path_to_movie_conversations, 'r') as file:
    lines = file.readlines()

  for line in lines:
    parts = line.replace('\n', '').split(' +++$+++ ')
    conversation = [line[1:-1] for line in parts[3][1:-1].split(', ')]

    for i in range(len(conversation) - 1):
      # 전처리 함수를 질문에 해당되는 inputs와 답변에 해당되는 outputs에 적용.
      inputs.append(preprocess_sentence(id2line[conversation[i]]))
      outputs.append(preprocess_sentence(id2line[conversation[i + 1]]))

      if len(inputs) >= MAX_SAMPLES:
        return inputs, outputs
  return inputs, outputs

In [141]:
# 한국어 데이터 전처리
data = origin_data.copy()
data = data[["Q", "A"]]
data["Q"] = data["Q"].apply(preprocess_sentence)
data["A"] = data["A"].apply(preprocess_sentence)
print('전체 샘플 수 :', len(data))

questions = data["Q"].values
answers = data["A"].values
print(questions[100])
print(answers[100])

전체 샘플 수 : 11823
거지됐어
밥 사줄 친구를 찾아 보세요


In [142]:
# 영어 데이터 전처리
en_questions, en_answers = load_conversations()
print('전체 샘플 수 :', len(en_questions))
print('전체 샘플 수 :', len(en_answers))

전체 샘플 수 : 50000
전체 샘플 수 : 50000


3) 단어장 만들기

In [152]:
import tensorflow_datasets as tfds

# 질문과 답변 데이터셋에 대해서 Vocabulary 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)
en_tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(en_questions + en_answers, target_vocab_size=2**13)

In [153]:
# 시작 토큰과 종료 토큰에 고유한 정수를 부여
START_TOKEN, END_TOKEN , DELIMITER_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1], [tokenizer.vocab_size + 2]
EN_START_TOKEN, EN_END_TOKEN , EN_DELIMITER_TOKEN = [en_tokenizer.vocab_size], [en_tokenizer.vocab_size + 1], [en_tokenizer.vocab_size + 2]

In [154]:
print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])
print('DELIMITER_TOKEN의 번호 :' ,[tokenizer.vocab_size + 2])
print('EN_START_TOKEN의 번호 :' ,[en_tokenizer.vocab_size])
print('EN_END_TOKEN의 번호 :' ,[en_tokenizer.vocab_size + 1])
print('EN_DELIMITER_TOKEN의 번호 :' ,[en_tokenizer.vocab_size + 2])

START_TOKEN의 번호 : [8354]
END_TOKEN의 번호 : [8355]
DELIMITER_TOKEN의 번호 : [8356]
EN_START_TOKEN의 번호 : [4964]
EN_END_TOKEN의 번호 : [4965]
EN_DELIMITER_TOKEN의 번호 : [4966]


In [162]:
# 시작 토큰과 종료 토큰을 고려하여 +3를 하여 단어장의 크기를 산정합니다.
VOCAB_SIZE = tokenizer.vocab_size + 3
print(VOCAB_SIZE)
EN_VOCAB_SIZE = en_tokenizer.vocab_size + 3
print(EN_VOCAB_SIZE)

8357
4967


2. 각 단어를 고유한 정수로 인코딩 & 패딩

In [163]:
# 임의의 22번째 샘플에 대해서 정수 인코딩 작업을 수행.
# 각 토큰을 고유한 정수로 변환
print('정수 인코딩 후의 21번째 질문 샘플: {}'.format(tokenizer.encode(questions[21])))
print('정수 인코딩 후의 21번째 답변 샘플: {}'.format(tokenizer.encode(answers[21])))
print('정수 인코딩 후의 21번째 질문 샘플: {}'.format(en_tokenizer.encode(questions[21])))
print('정수 인코딩 후의 21번째 답변 샘플: {}'.format(en_tokenizer.encode(answers[21])))

정수 인코딩 후의 21번째 질문 샘플: [5821, 606, 2493, 4172]
정수 인코딩 후의 21번째 답변 샘플: [2679, 7662, 8, 6365, 95, 1]
정수 인코딩 후의 21번째 질문 샘플: [4942, 4884, 4836, 4944, 4846, 4872, 4943, 4893, 4840, 4740, 4944, 4866, 4873, 4943, 4838, 4864, 4740, 4944, 4857, 4840, 4943, 4847, 4860]
정수 인코딩 후의 21번째 답변 샘플: [4943, 4847, 4872, 4944, 4865, 4848, 4740, 4943, 4847, 4880, 4944, 4859, 4852, 4943, 4846, 4856, 4740, 4943, 4849, 4856, 4740, 4944, 4868, 4844, 4944, 4857, 4897, 4945, 4857, 4888, 4943, 4888, 4852, 4944, 4862, 4856, 7]


In [164]:
# 샘플의 최대 허용 길이 또는 패딩 후의 최종 길이
MAX_LENGTH = 80
print(MAX_LENGTH)

80


In [165]:
# 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
def tokenize_and_filter(inputs, outputs, tokenizer, special_tokens):
  tokenized_qa_inputs, tokenized_a_inputs = [], []
  start_token, delimiter_token, end_token = special_tokens
  for (sentence1, sentence2) in zip(inputs, outputs):
    # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
    # Question Answer input 구성
    sentence1 = start_token + tokenizer.encode(sentence1) + delimiter_token + tokenizer.encode(sentence2) + end_token
    # Answer label 구성
    sentence2 = delimiter_token + tokenizer.encode(sentence2) + end_token

    # 최대 길이 MAX_LENGTH 이하인 경우에만 데이터셋으로 허용
    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_qa_inputs.append(sentence1)
      tokenized_a_inputs.append(sentence2)
  
  # 최대 길이 40으로 모든 데이터셋을 패딩
  tokenized_qa_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_qa_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_a_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_a_inputs, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_qa_inputs, tokenized_a_inputs

In [166]:
# 한국어 인코딩
question_answer_inputs, answers_inputs = tokenize_and_filter(questions, answers, tokenizer, [START_TOKEN, DELIMITER_TOKEN, END_TOKEN])
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 질문 샘플 개수: {}'.format(len(questions)))
print('필터링 후의 답변 샘플 개수: {}'.format(len(answers)))

단어장의 크기 : 8357
필터링 후의 질문 샘플 개수: 11823
필터링 후의 답변 샘플 개수: 11823


In [167]:
# 영어 인코딩
en_question_answer_inputs, en_answers_inputs = tokenize_and_filter(en_questions, en_answers, en_tokenizer, [EN_START_TOKEN, EN_DELIMITER_TOKEN, EN_END_TOKEN])
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 질문 샘플 개수: {}'.format(len(en_questions)))
print('필터링 후의 답변 샘플 개수: {}'.format(len(en_answers)))

단어장의 크기 : 8357
필터링 후의 질문 샘플 개수: 50000
필터링 후의 답변 샘플 개수: 50000


3. Teaching forcing

In [168]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더는 이전의 target을 다음의 input으로 사용합니다.
# 이에 따라 outputs에서는 START_TOKEN을 제거하겠습니다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': question_answer_inputs,
    },
    {
        'outputs': answers_inputs
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

en_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': en_question_answer_inputs,
    },
    {
        'outputs': en_answers_inputs
    },
))

en_dataset = en_dataset.cache()
en_dataset = en_dataset.shuffle(BUFFER_SIZE)
en_dataset = en_dataset.batch(BATCH_SIZE)
en_dataset = en_dataset.prefetch(tf.data.experimental.AUTOTUNE)

## 3. GPT 모델링

1) Scaled dot product Attention

In [82]:
# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product_attention(query, key, value, mask):
  # 어텐션 가중치는 Q와 K의 닷 프로덕트
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 가중치를 정규화
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 패딩에 마스크 추가
  if mask is not None:
    logits += (mask * -1e9)

  # softmax적용
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # 최종 어텐션은 가중치와 V의 닷 프로덕트
  output = tf.matmul(attention_weights, value)
  return output

2) Multi-Head Attention

In [83]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # Q, K, V에 각각 Dense를 적용합니다
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # 병렬 연산을 위한 머리를 여러 개 만듭니다
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # 스케일드 닷 프로덕트 어텐션 함수
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 최종 결과에도 Dense를 한 번 더 적용합니다
    outputs = self.dense(concat_attention)

    return outputs

3) padding mask

In [84]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]

4) look ahead mask

In [85]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

5) Decoder Layer
* inputs 으로 encoder 의 ouputs 을 받지 않음

In [86]:
# 디코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 세 개의 서브 레이어가 존재합니다.
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    look_ahead_mask = tf.keras.Input(
        shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
    attention1 = MultiHeadAttention(
        d_model, num_heads, name="attention_1")(inputs={
            'query': inputs,
            'key': inputs,
            'value': inputs,
            'mask': look_ahead_mask
        })

    # 마스크드 멀티 헤드 어텐션의 결과는
    # Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
    attention1 = tf.keras.layers.Dropout(rate=dropout)(attention1)
    attention1 = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(attention1)

    # 세 번째 서브 레이어 : 2개의 완전연결층
    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention1)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)

    # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(outputs + attention1)

    return tf.keras.Model(
        inputs=[inputs, look_ahead_mask, padding_mask], # enc_outputs 제외
        outputs=outputs,
        name=name)

6) Decoder Module
* encoder outputs 받는 부분 제외
* positional encoding 대신 positinal embedding layer로 변경

In [87]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    look_ahead_mask = tf.keras.Input(
        shape=(1, None, None), name='look_ahead_mask')

    # 패딩 마스크
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    # 임베딩 레이어
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    
    # potision
    # input token Number 
    positions = tf.keras.layers.Lambda(
        lambda i: tf.shape(i)[-1],
        output_shape=(1,),
        name='n_tokens')(inputs)
    position_embeddings = tf.keras.layers.Embedding(MAX_LENGTH, d_model)(positions)

    # 포지셔널 인코딩 제외
    # embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    # Dropout이라는 훈련을 돕는 테크닉을 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings + position_embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
        inputs=[inputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name)

7) GPT Model

In [191]:
def gpt(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="GPT"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")

    # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용합니다.
    # 내부적으로 패딩 마스크도 포함되어져 있습니다.
    look_ahead_mask = tf.keras.layers.Lambda(
        create_look_ahead_mask,
        output_shape=(1, None, None),
        name='look_ahead_mask')(inputs)

    # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
    # 디코더에서 패딩을 위한 마스크
    dec_padding_mask = tf.keras.layers.Lambda(
        create_padding_mask, output_shape=(1, 1, None),
        name='dec_padding_mask')(inputs)

    # 디코더
    dec_outputs = decoder(
        vocab_size=vocab_size,
        num_layers=num_layers,
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
    )(inputs=[inputs, look_ahead_mask, dec_padding_mask])

    # 완전연결층
    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs], outputs=outputs, name=name)

In [192]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 4 # 디코더의 층의 개수
D_MODEL = 256 # 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model = gpt(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

Model: "GPT"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
look_ahead_mask (Lambda)        (None, 1, None, None 0           inputs[0][0]                     
__________________________________________________________________________________________________
dec_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
decoder (Functional)            (None, None, 256)    4268288     inputs[0][0]                     
                                                                 look_ahead_mask[0][0]          

8) Loss Function

In [106]:
def loss_function(y_true, y_pred):

    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH))

    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

9) Custom Scheduler

In [107]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

## 4. GPT 모델 훈련

In [108]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [109]:
# 한국어 훈련
EPOCHS = 20
model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/50
185/185 [==============================] - 27s 112ms/step - loss: 0.8046 - accuracy: 0.0103
Epoch 2/50
185/185 [==============================] - 20s 110ms/step - loss: 0.6423 - accuracy: 0.0174
Epoch 3/50
185/185 [==============================] - 20s 108ms/step - loss: 0.5413 - accuracy: 0.0265
Epoch 4/50
185/185 [==============================] - 20s 108ms/step - loss: 0.5190 - accuracy: 0.0271
Epoch 5/50
185/185 [==============================] - 20s 109ms/step - loss: 0.5107 - accuracy: 0.0275
Epoch 6/50
185/185 [==============================] - 20s 110ms/step - loss: 0.5051 - accuracy: 0.0278
Epoch 7/50
185/185 [==============================] - 20s 109ms/step - loss: 0.5005 - accuracy: 0.0280
Epoch 8/50
185/185 [==============================] - 20s 108ms/step - loss: 0.4955 - accuracy: 0.0283
Epoch 9/50
185/185 [==============================] - 20s 109ms/step - loss: 0.4911 - accuracy: 0.0287
Epoch 10/50
185/185 [==============================] - 20s 109ms/step - l

In [169]:
# 영어 훈련
EPOCHS = 20
model.fit(en_dataset, epochs=EPOCHS, verbose=1)

Epoch 1/20
782/782 [==============================] - 85s 109ms/step - loss: 0.1621 - accuracy: 0.0267
Epoch 2/20
782/782 [==============================] - 85s 108ms/step - loss: 0.1494 - accuracy: 0.0282
Epoch 3/20
782/782 [==============================] - 85s 108ms/step - loss: 0.1482 - accuracy: 0.0284
Epoch 4/20
782/782 [==============================] - 85s 108ms/step - loss: 0.1476 - accuracy: 0.0284
Epoch 5/20
782/782 [==============================] - 85s 108ms/step - loss: 0.1471 - accuracy: 0.0284
Epoch 6/20
612/782 [======================>.......] - ETA: 18s - loss: 0.1483 - accuracy: 0.0283

KeyboardInterrupt: 

영어 데이터셋 학습이 제대로 되지 않아 중단, 모델 수정 후 재 학습

In [193]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 3 # 디코더의 층의 개수
D_MODEL = 256 # 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

en_model = gpt(
    vocab_size=EN_VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

en_model.summary()

Model: "GPT"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
look_ahead_mask (Lambda)        (None, 1, None, None 0           inputs[0][0]                     
__________________________________________________________________________________________________
dec_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
decoder (Functional)            (None, None, 256)    2873344     inputs[0][0]                     
                                                                 look_ahead_mask[0][0]          

In [172]:
en_model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [174]:
# 영어 훈련
EPOCHS = 5
en_model.fit(en_dataset, epochs=EPOCHS, verbose=1)

Epoch 1/5
782/782 [==============================] - 46s 59ms/step - loss: 0.1459 - accuracy: 0.0288
Epoch 2/5
782/782 [==============================] - 46s 59ms/step - loss: 0.1457 - accuracy: 0.0288
Epoch 3/5
782/782 [==============================] - 46s 59ms/step - loss: 0.1456 - accuracy: 0.0289
Epoch 4/5
782/782 [==============================] - 46s 59ms/step - loss: 0.1453 - accuracy: 0.0289
Epoch 5/5
782/782 [==============================] - 46s 59ms/step - loss: 0.1455 - accuracy: 0.0290


## 5. 테스트

In [186]:
def decoder_inference(sentence, model, tokenizer, special_tokens):
    sentence = preprocess_sentence(sentence)
    start_token, delimiter_token, end_token = special_tokens

    # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
    sentence = tf.expand_dims(
        start_token + tokenizer.encode(sentence) + delimiter_token, axis=0)

    # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
    output_sequence = tf.expand_dims(delimiter_token, axis=0)

    # 디코더의 인퍼런스 단계
    for i in range(MAX_LENGTH):
        # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
        predictions = model(inputs=[sentence], training=False)
        predictions = predictions[:, -1:, :]

        # 현재 예측한 단어의 정수
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
        if tf.equal(predicted_id, END_TOKEN[0]):
          print(predicted_id)
          break
        
        # 예측 단어 추가
        output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

    return tf.squeeze(output_sequence, axis=0)

In [183]:
def sentence_generation(sentence, model, tokenizer, special_tokens):
  # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
  prediction = decoder_inference(sentence, model, tokenizer, special_tokens)

  # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('입력 : {}'.format(sentence))
  print('출력 : {}'.format(predicted_sentence))

  return predicted_sentence

In [187]:
sentence_generation('무서운 영화 어때?', 
                    model=model, 
                    tokenizer=tokenizer, 
                    special_tokens=[START_TOKEN, DELIMITER_TOKEN, END_TOKEN])

입력 : 무서운 영화 어때?
출력 :  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .


' . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .'

In [188]:
sentence_generation('Where have you been?', 
                    model=en_model, 
                    tokenizer=en_tokenizer, 
                    special_tokens=[EN_START_TOKEN, EN_DELIMITER_TOKEN, EN_END_TOKEN])

입력 : Where have you been?
출력 : 


''

## 회고
* 배운 점
  - GPT 모델의 상세한 구현에 대해 학습할 수 있었습니다.
* 아쉬운 점
  - 데이터가 적어서 그런건지 제대로 학습이 안되는 것 같습니다.
* 느낀 점
  - 언어모델에 대해서 재미를 조금 느꼈습니다.
* 어려웠던 점
  - 논문의 내용중 L3 objective 까지는 구현을 못했습니다.